In [1]:
import numpy as np
import pandas as pd
import copy

from hongxiongmao import utilities
from hongxiongmao import download
from hongxiongmao import overplot
dl = download.quandl_hxm()

import plotly.offline as py
import plotly_express as pyex
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)

In [9]:
us_recession_watch = {
    'Recession_Probability':dict(ticker='FRED/RECPROUSM156N', name=''),
    'ISM':dict(ticker='ISM/MAN_PMI', name='ISM PMI Composite'),
    '10yr':dict(ticker='USTREASURY/YIELD', fields=['10 YR'], name='ISM PMI Composite')
    }

In [10]:
dl.from_tickerdict(us_recession_watch)

,FRED/RECPROUSM156N - Value,ISM/MAN_PMI - PMI,USTREASURY/YIELD - 1 MO,USTREASURY/YIELD - 2 MO,USTREASURY/YIELD - 3 MO,USTREASURY/YIELD - 6 MO,USTREASURY/YIELD - 1 YR,USTREASURY/YIELD - 2 YR,USTREASURY/YIELD - 3 YR,USTREASURY/YIELD - 5 YR,USTREASURY/YIELD - 7 YR,USTREASURY/YIELD - 10 YR,USTREASURY/YIELD - 20 YR,USTREASURY/YIELD - 30 YR
Date,,,,,,,,,,,,,,
2018-08-31,NaN,NaN,1.95,NaN,2.11,2.28,2.46,2.62,2.70,2.74,2.81,2.86,2.95,3.02
2018-09-30,0.12,59.5,2.12,NaN,2.19,2.36,2.59,2.81,2.88,2.94,3.01,3.05,3.13,3.19
2018-10-31,0.08,57.5,2.20,2.26,2.34,2.49,2.69,2.87,2.93,2.98,3.07,3.15,3.30,3.39
2018-11-30,0.08,58.8,2.31,2.33,2.37,2.52,2.70,2.80,2.83,2.84,2.92,3.01,3.19,3.30
2018-12-31,0.12,54.3,2.44,2.45,2.45,2.56,2.63,2.48,2.46,2.51,2.59,2.69,2.87,3.02
2019-01-31,0.38,56.6,2.42,2.43,2.41,2.46,2.55,2.45,2.43,2.43,2.51,2.63,2.83,2.99
2019-02-28,0.96,54.2,2.44,2.47,2.45,2.50,2.54,2.52,2.50,2.52,2.63,2.73,2.94,3.09
2019-03-31,0.62,55.3,2.43,2.44,2.40,2.44,2.40,2.27,2.21,2.23,2.31,2.41,2.63,2.81
2019-04-30,0.82,52.8,2.43,2.44,2.43,2.46,2.39,2.27,2.24,2.28,2.39,2.51,2.75,2.93
